In [17]:
#!pip install trafilatura, bs4
"""
On my mac at least i needed to
#pip install -U charset_normalizer
#pip uninstall charset_normalizer -y
#!pip install lxml_html_clean
"""
import requests
import trafilatura
from bs4 import BeautifulSoup

def fetch_text(uri):
    return trafilatura.extract(trafilatura.fetch_url(uri))

"""the simplest possible text scrape"""
site = 'https://docs.fastht.ml/'
downloaded = trafilatura.fetch_url(site)
home_content = trafilatura.extract(downloaded)
soup = BeautifulSoup(downloaded, 'html.parser')
links = soup.find_all('a')
all_links = set([link.get('href') for link in links if link.get('href','-')[0]=='.'])
all_links = [f"{site}{l[2:]}" for l in all_links]
all_pages = {uri: fetch_text(uri) for uri in all_links }
home_content

'FastHTML\nWelcome to the official FastHTML documentation.\nFastHTML is a new next-generation web framework for fast, scalable web applications with minimal, compact code. It’s designed to be:\n- Powerful and expressive enough to build the most advanced, interactive web apps you can imagine.\n- Fast and lightweight, so you can write less code and get more done.\n- Easy to learn and use, with a simple, intuitive syntax that makes it easy to build complex apps quickly.\nFastHTML apps are just Python code, so you can use FastHTML with the full power of the Python language and ecosystem. FastHTML’s functionality maps 1:1 directly to HTML and HTTP, but allows them to be encapsulated using good software engineering practices—so you’ll need to understand these foundations to use this library fully. To understand how and why this works, please read this first: about.fastht.ml.\nInstallation\nSince fasthtml\nis a Python library, you can install it with:\npip install python-fasthtml\nIn the near

## Create a lancedb dump of the data (not attempt to chunk)

In [52]:
import pydantic,lancedb
from lancedb.pydantic import LanceModel, Vector
from pydantic import BaseModel
import numpy as np
from lancedb.embeddings import get_registry

"""connect to the db (file)"""
db = lancedb.connect('../docs_search/app/data/')


"""https://lancedb.github.io/lancedb/embeddings/#openai-embedding-function"""
func = get_registry().get("openai").create(name="text-embedding-ada-002")

class HelpContent(LanceModel):
    """a simple lance data model to wrap the embedding in IO"""
    id: str
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()
        
"""Create a data model for content"""

data = [HelpContent(id=uri,vector=np.random.randn(1536), text=content) 
        for uri, content in all_pages.items()]

table = db.create_table("fast_html_help", 
                        schema=HelpContent, 
                        mode="overwrite", 
                        data=data)
tbl = db.open_table('fast_html_help')
tbl.to_pandas()

,id,text,vector
0,https://docs.fastht.ml/explains/oauth.html,OAuth\nOAuth is an open standard for ‘access d...,"[-1.0064234, 2.2734828, -1.2928158, -1.3717712..."
1,https://docs.fastht.ml/api/oauth.html,It can sometimes be useful to pass state to th...,"[-0.8965331, 0.7940203, -0.16225068, 1.0325359..."
2,https://docs.fastht.ml/explains/faq.html,FAQ\nFrequently Asked Questions\nWhy is FastHT...,"[1.1236138, -1.5185752, 1.2172407, -0.38552383..."
3,https://docs.fastht.ml/ref/live_reload.html,Live Reloading\nWhen building your app it can ...,"[0.69199264, -1.7246873, -1.5288838, 0.3589086..."
4,https://docs.fastht.ml/api/components.html,from lxml import html as lx\nfrom pprint impor...,"[-2.2917068, -0.11244544, -0.5166978, -0.78804..."
5,https://docs.fastht.ml/explains/routes.html,from fasthtml.common import *\nRoutes\nBehavio...,"[-0.4551263, -1.4703597, 1.1597834, 0.04954685..."
6,https://docs.fastht.ml/tutorials/index.html,Tutorials\nClick through to any of these tutor...,"[-0.0495438, -0.5392144, 0.94179493, -0.286452..."
7,https://docs.fastht.ml/api/js.html,'.body {color: green;}') light_media(\n<style>...,"[-1.1749271, -0.55591196, 0.90942967, -0.75545..."
8,https://docs.fastht.ml/explains/explaining_xt_...,from fasthtml.common import *\ndef example():\...,"[-1.1048688, 0.6548778, -0.17273702, 1.1018617..."
9,https://docs.fastht.ml/api/core.html,import time\nfrom IPython import display\nfrom...,"[0.061107706, 0.08670035, 0.8202128, 1.1363777..."


In [53]:
tbl.search('is there a JS App Walkthrough').select(['text']).to_list()

[{'text': '# Create some dummy posts\nimport pathlib\n"posts").mkdir(exist_ok=True)\npathlib.Path(for i in range(10):\nf"posts/article_{i}.md").write_text(f"This is article {i}") pathlib.Path(\nBYO Blog\nThis document is a work in progress.\nIn this tutorial we’re going to write a blog by example. Blogs are a good way to learn a web framework as they start simple yet can get surprisingly sophistated. The wikipedia definition of a blog is “an informational website consisting of discrete, often informal diary-style text entries (posts) informal diary-style text entries (posts)”, which means we need to provide these basic features:\n- A list of articles\n- A means to create/edit/delete the articles\n- An attractive but accessible layout\nWe’ll also add in these features, so the blog can become a working site:\n- RSS feed\n- Pages independent of the list of articles (about and contact come to mind)\n- Import and Export of articles\n- Tagging and categorization of data\n- Deployment\n- Abil